# Week 12 Problem Set

## Homework

**HW1.** *Comments:* Write a state machine whose inputs are the characters of a string. The string contains the code for a computer program. The output of the state machine are either:
- the input character if it is part of a comment, or
- `None`, otherwise.

Comment starts with a `#` character and continue to the end of the current line. If you want to create a string that contains a new line character, you can use `\n`.

For example,
```
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
print(m.transduce(inpstr))
```

The expected output is:
```
[None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]
```

You should start by drawing a state transition diagram indicating the states and what inputs cause transition to which other states. Use the test case above to determine if your state transition diagram is correct. You should begin writing your program only when you are confident that your diagram is correct. 

In [1]:
# Copy over the implementation of StateMachine from Cohort
from abc import ABC, abstractmethod

class StateMachine(ABC):
    
    def start(self):
        self.state = self.start_state

    def step(self, inp):
        ns, o = self.get_next_values(self.state, inp)
        self.state = ns
        return o
        
    def transduce(self, inp_list):
        output = []
        self.start()
        for val in inp_list:
            temp = self.step(val)
            output.append(temp)
            if self.is_done():
                break
        return output

    @abstractmethod
    def get_next_values(self, state, inp):
        pass

    def done(self, state):
        return False

    def is_done(self):
        return self.done(self.state)

In [2]:
class CommentsSM(StateMachine):
    
    def __init__(self):
        self.start_state = 0

    def get_next_values(self, state, inp):
        if state == 0:
            next_state = 0
            output = None
            if inp == '#':
                next_state = 1
                output = inp
        if state == 1:
            next_state = 1
            output = inp
            if inp == '\n':
                next_state = 0
                output = None
        return next_state, output
    
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
out = m.transduce(inpstr)

In [3]:
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
out = m.transduce(inpstr)
assert out == [None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]

In [4]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** *First Word:* Write a state machine whose inputs are the characters of a string and which outputs either:
- the input character if it is part of the first word on a line, or
- `None`, otherwise

For the purposes here, a word is any sequence of consecutive characters that does not contain spaces or end-of-line characters. In this problem, comments have no special status. This means that if the line begins with `# `, then the first word is `#`. 

For example, 
```
inpstr = "def func(x): # comment\n    return 1"
m = FirstWordSM()
print(m.transduce( inpstr))
```

The expected output is:
```
["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]
```

In [5]:
class FirstWordSM(StateMachine):
  
    def __init__(self):
        self.start_state = 0
  
    def get_next_values(self, state, inp):
        if state == 0:
            next_state = 0
            output = inp
            if inp == ' ':
                next_state = 1
                output = None
            if inp == '\n':
                next_state = 2
                output = None
        if state == 1:
            next_state = 1
            output = None
            if inp == '\n':
                next_state = 2
        if state == 2:
            next_state = 2
            output = None
            if inp != ' ':
                next_state = 0
                output = inp
        return next_state, output
    
inpstr = "def func(x): # comment\n  pass\n    return 1"
m = FirstWordSM()
print(m.transduce( inpstr))

['d', 'e', 'f', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'p', 'a', 's', 's', None, None, None, None, None, 'r', 'e', 't', 'u', 'r', 'n', None, None]


In [6]:
inpstr = "def func(x): # comment\n    return 1"
m = FirstWordSM()
out = m.transduce(inpstr)
assert out == ["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]


In [7]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Robot:* Write a State Machine class that represent a robot. The dimension of the world and the robot initial position should be specified during the class instantiation. The robot can take in the following input:
- "left"
- "right"
- "up"
- "down"

The initial position of the robot is specified during the object instantiation and the input should modify the position of the robot. The robot position must not change if it exceed the boundary. At each step, the robot should output the updated position. 

In [8]:
class Position:
  
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
  
    def __str__(self):
        return f"({self.x:}, {self.y:})"

class Dimension:

    def __init__(self, width=0, height=0):
        self.width = width
        self.height = height
  
    def __str__(self):
        return f"width: {self.width:}, height: {self.height:}"

In [9]:
class RobotSM(StateMachine):

    def __init__(self, init_pos, dimension):
        self.world_dim = dimension
        self.start_state = init_pos
  
    def get_next_values(self, state, inp):
        if inp == 'right' and state.x < self.world_dim.width:
            state.x += 1
        if inp == 'up' and state.y < self.world_dim.height:
            state.y += 1
        if inp == 'left' and state.x < self.world_dim.width:
            state.x -= 1
        if inp == 'down' and state.y < self.world_dim.height:
            state.y -= 1
        next_state = state
        output = next_state
        return next_state, output

In [10]:
robot = RobotSM(Position(0, 0), Dimension(5, 5))
robot.start()
robot.transduce(["right", "right", "up", "up", "up", "left", "down"])
pos = robot.state
assert pos.x == 1 and pos.y == 2

In [11]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Search SM:* Write a function `sm_search` that takes in the following arguments:
- `sm_to_search`: is the State Machine instance to search. This argument is of the type `MapSM` as defined in CS4. You should use the `get_next_values()` of this State Machine instance to explore the next state in your search.
- `initial_state`: is the start state of the search. If it is not provided, it should be assigned to the `start_state` of sm_search. 
- `goal_test`: is a function that returns `True` if the argument is the end state of the search. If it is not provided, it should be eassigned to the `done` function of the state machine.

This function performs a **breadth-first-search** algorithm to explore the next states. 

The output is a `list` of `Step` instances from the `init_state` to the end state which is determined by the `goal_test` function.

This problem requires you to complete the following:
- `Queue` class from Problem Set 4 HW2.
- `MapSM` class in CS4.
- `SearchNode` and `Step` classes in CS5.

In [12]:
# Copy over the implementations of Queue from PS4 HW2
class Stack:
    def __init__(self):
        self._items = []
        
    def push(self, item):
        self._items.append(item)

    def pop(self):
        if self.is_empty:
            return None
        else:
            item = self._items[-1]
            del self._items[-1]
            return item

    def peek(self):
        if self.is_empty:
            return None
        else:
            return self._items[-1]

    @property
    def is_empty(self):
        if self._items == []:
            return True
        else:
            return False

    @property
    def size(self):
        return len(self._items)
    
class Queue:
    def __init__(self):
        self.left_stack = Stack()
        self.right_stack = Stack()
    
    @property
    def is_empty(self):
        return self.left_stack.is_empty and self.right_stack.is_empty
        
    @property
    def size(self):
        return self.left_stack.size + self.right_stack.size
    
    def right_to_left(self):
        while not self.right_stack.is_empty:
            item = self.right_stack.pop()
            self.left_stack.push(item)
        
    def enqueue(self, item):
        self.right_stack.push(item)
        
    def dequeue(self):
        if self.left_stack.is_empty:
            self.right_to_left()
        return self.left_stack.pop()
                
    def peek(self):
        if self.left_stack.is_empty:
            self.right_to_left()
        return self.left_stack.peek()


In [13]:
# Copy over the implementation of StateSpaceSearch from Cohort
from abc import abstractmethod

class StateSpaceSearch(StateMachine):
    @property
    @abstractmethod
    def statemap(self):
        pass

    @property
    @abstractmethod
    def legal_inputs(self):
        pass


In [14]:
# Copy over the implementation of MapSM from Cohort
class MapSM(StateSpaceSearch):
    
    def __init__(self, start):
        self.start_state = start
        
    @property
    def statemap(self):
        statemap = {"S": ["A", "B"],
                    "A": ["S", "C", "D"],
                    "B": ["S", "D", "E"],
                    "C": ["A", "F"],
                    "D": ["A", "B", "F", "H"],
                    "E": ["B", "H"],
                    "F": ["C", "D", "G"],
                    "H": ["D", "E", "G"],
                    "G": ["F", "H"]}
        return statemap

    @property
    def legal_inputs(self):
        max_len = 0
        for node, neighbours in self.statemap.items():
            length = len(neighbours)
            if length > max_len:
                max_len = length
        out = set(range(max_len))
        return out

  
#     def get_next_values(self, state, inp):
#         neighbours = self.statemap.
#         if neighbours == None or inp >= len(neighbours):
#             next_state = state
#         else:
            
#             next_state = neighbours[inp]
#         output = next_state
#         return next_state, output
    def get_next_values(self, state, inp):
        neighbours = self.statemap[state]
        if neighbours == [] or inp >= len(neighbours):
            next_state = state
        else:
            next_state = neighbours[inp]
        output = next_state
        return next_state, output


In [17]:
# Copy over the implementations of Step and SearchNode from Cohort
class Step:
    def __init__(self, action, state):
        self.action = action
        self.state = state
    
    def __eq__(self, other):
        return self.action == other.action and self.state == other.state
  
    def __str__(self):
        return f"action: {self.action:}, state: {self.state:}"

class SearchNode:
    def __init__(self, action, state, parent):
        self.state = state
        self.action = action
        self.parent = parent
  
    def path(self):
        output = []
        if self.parent == None:
            return [Step(self.action, self.state)]
        else:
            temp = SearchNode(self.parent.action, self.parent.state, self.parent.parent).path()
            return temp + [Step(self.action, self.state)]
  
    def in_path(self, state):
        if self.state == state:
            return True
        elif self.parent == None:
            return False
        else:
            return self.parent.in_path(state)
  
    def __eq__(self, other):
        if self is None and other is None:
            return True
        elif self is None:
            return False
        elif other is None:
            return False
        else:
            return self.state == other.state and self.parent == other.parent and \
                   self.action == other.action

In [18]:
def sm_search(sm_to_search, initial_state=None, goal_test=None):
    # check if initial_state is provided
    # if it is, use it
    # otherwise, get the start state of sm_to_search
    if initial_state == None:
        # replace None to take the start state of sm_to_search
        init_state = sm_to_search.start_state
    else:
        init_state = initial_state
  
    # check if goal_test is provided
    # if it is, use it
    # otherwise, use the done method as the goal function
    # taken from sm_to_search
    if goal_test == None:
        goal_func = sm_to_search.done
    else:
        goal_func = goal_test
  
    # create a Queue instance to store the node to explore
    # replace the None below
    agenda = Queue()
  
    # if the initial state is the goal state, 
    # then we are done and exit
    if goal_func(init_state):
        return [Step(None, init_state)]
  
    # otherwise, add the current node into the agenda 
    agenda.enqueue(SearchNode(None, init_state, None))
    
    # explore as long as the Queue is not empty
    while not agenda.is_empty:
        
        # replace None to take out the parent from the Queue
        parent = agenda.dequeue()
        
        # create a list to keep track which child state have been explored
        new_child_state = []
        
        # get all the legal input values
        actions = sm_to_search.legal_inputs
        
        #iterate over all legal inputs
        for a in actions:
            # get the next possible state using the current action
            # call get_next_values to get the next state
            # replace the None below
            new_s = sm_to_search.get_next_values(parent.state, a)[0]
            
            # create a new search node from the new_s
            # replace the None below
            new_n = SearchNode(a, new_s, parent)
            
            # if the new state is the goal state, then we exit and return the path
            if goal_func(new_s):
                return new_n.path()
            
            # if the new state is already in the list of new child state, ignore it
            elif new_s in new_child_state:
                pass
            
            # if the new state is in the path of the current node, ignore it
            elif parent.in_path(new_s):
                pass
            
            # otherwise, add the new state into the list
            # and the new node into the Queue
            else:
                # step 1. add the new state into the new_child_state
                new_child_state.append(new_s)
                
                # step 2. add the new node into the Queue
                agenda.enqueue(new_n)
    return new_child_state

mapSM = MapSM("S")
ans = sm_search(mapSM , "S" , lambda s: s=="H" )
steps = [(step.action, step.state) for step in ans]
print(ans)

[<__main__.Step object at 0x7f70f1f35910>, <__main__.Step object at 0x7f70f1f358d0>, <__main__.Step object at 0x7f70f1f35890>, <__main__.Step object at 0x7f70f1f35850>]


In [19]:
mapSM = MapSM("S")
ans = sm_search(mapSM , "S" , lambda s: s=="H" )
steps = [(step.action, step.state) for step in ans]
assert steps == [(None, "S"), (0, "A"), (2, "D"), (3, "H")]
for step in ans:
    print(step)

action: None, state: S
action: 0, state: A
action: 2, state: D
action: 3, state: H


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
